<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=88fa8ea76e33da20d43849f1938d94fda856372ee738831665ebc2cf9372846b
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 10:40:17
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.44 L
Current:  1.43 C
-------------------
Today PnL: 50.04 K (0.35%)
Current PnL: -16.37 L (-11.01%)
CY Booked + Current PnL: -5.09 L (-3.42%)
-------------------
Total profit:  2.28 L
Total loss:  -18.65 L
-------------------
Total Booked + Current PnL: 21.89 L (17.99%)
Total Booked PnL: 38.26 L (31.44%)
Curr Year Booked PnL: 11.28 L (7.88%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 81.99 L (57.31%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 10.03%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.86,55.0,H-LC,10.84,172216.0,16039.0,10057.0,-0.76,10.27,5.84,16.71,22.0,1.59,1.24,29.50,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,104.70,58.0,M-SC,3.97,88196.0,-12581.0,12674.0,-0.52,-12.48,14.37,0.09,245.0,-0.99,0.64,15.87,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,65.0,H-MC,6.84,113309.0,12805.0,17835.0,0.25,12.74,15.74,30.49,79.0,0.72,0.82,49.27,MH,ATH,METALS
57,RELIANCE,1533.00,-11.79,60.0,H-LC,5.19,220849.0,10483.0,18308.0,0.13,4.98,8.29,13.68,37.0,0.57,1.59,22.33,XY25,NTT,REFINERIES
42,ITC,452.00,-37.50,52.0,H-LC,2.52,201346.0,1208.0,19692.0,0.60,0.60,9.78,10.44,4.0,0.06,1.45,6.20,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.46,57.0,H-SC,17.06,201840.0,1886.0,95349.0,0.20,0.94,47.24,48.63,115.0,0.02,1.46,12.61,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.65,45.0,H-MC,2.23,136459.0,671.0,42084.0,0.62,0.49,30.84,31.48,107.0,0.02,0.98,12.67,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.34,59.0,H-MC,1.88,248602.0,871.0,74183.0,0.01,0.35,29.84,30.30,92.0,0.01,1.79,13.69,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.68,53.0,H-MC,4.20,200700.0,-120.0,68900.0,-0.03,-0.06,34.33,34.25,80.0,-0.00,1.45,5.13,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-17.03,39.0,H-LC,6.39,298584.0,-17258.0,129914.0,-0.65,-5.46,43.51,35.67,5.0,-0.13,2.15,7.63,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-12.93,47.0,H-LC,7.84,219639.0,-32129.0,85330.0,0.09,-12.76,38.85,21.13,27.0,-0.38,1.58,18.14,X40,ATH,PAINTS
42,ITC,452.00,-37.50,52.0,H-LC,2.52,201346.0,1208.0,19692.0,0.60,0.60,9.78,10.44,4.0,0.06,1.45,6.20,X40,NTT,FMCG
44,JIOFIN,387.00,0.39,53.0,H-LC,13.23,272835.0,5470.0,59996.0,0.44,2.05,21.99,24.48,48.0,0.09,1.97,58.14,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-12.66,53.0,H-LC,7.78,228932.0,-5212.0,53845.0,-0.46,-2.23,23.52,20.77,86.0,-0.10,1.65,4.54,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,0.87,72.0,H-LC,1.29,250268.0,4117.0,75080.0,2.55,1.67,30.00,32.18,16.0,0.05,1.81,38.32,X200,ATH,IT
42,ITC,452.0,-37.50,52.0,H-LC,2.52,201346.0,1208.0,19692.0,0.60,0.60,9.78,10.44,4.0,0.06,1.45,6.20,X40,NTT,FMCG
51,NESTLEIND,1377.0,-8.35,57.0,H-LC,4.03,282204.0,16778.0,40017.0,0.14,6.32,14.18,21.40,11.0,0.42,2.04,14.11,XY25,NTT,FMCG
57,RELIANCE,1533.0,-11.79,60.0,H-LC,5.19,220849.0,10483.0,18308.0,0.13,4.98,8.29,13.68,37.0,0.57,1.59,22.33,XY25,NTT,REFINERIES
20,CIPLA,1795.0,-19.78,53.0,H-LC,5.77,214474.0,9974.0,31442.0,0.38,4.88,14.66,20.25,10.0,0.32,1.55,14.04,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LAOPALA,464.00,118.89,56.0,H-SC,2.33,100199.0,-30906.0,82153.0,2.96,-23.57,81.99,39.09,142.0,-0.38,0.72,29.05,AR,NTT,CERAMICS
48,LTIM,7230.20,0.87,72.0,H-LC,1.29,250268.0,4117.0,75080.0,2.55,1.67,30.00,32.18,16.0,0.05,1.81,38.32,X200,ATH,IT
11,BANDHANBNK,400.00,-9.90,54.0,H-SC,5.13,225120.0,-53439.0,310891.0,2.26,-19.18,138.10,92.42,151.0,-0.17,1.62,30.10,XY24,NTT,BANKS
8,ATULAUTO,844.00,3948.65,65.0,M-SC,5.79,126201.0,-21167.0,72982.0,2.12,-14.36,57.83,35.16,236.0,-0.29,0.91,28.90,XY24,NTT,AUTO
16,BSOFT,836.99,-12.08,58.0,H-SC,15.02,98940.0,-41713.0,114493.0,1.84,-29.66,115.72,51.74,131.0,-0.36,0.71,12.83,XR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,12.74,89139.0,-11824.0,106352.0,-0.99,-11.71,119.31,93.63,148.0,-0.11,0.64,32.77,SR,ATH,CHEMICALS
70,SYMPHONY,1306.00,-41.90,40.0,M-SC,18.29,123887.0,-47254.0,47201.0,-0.79,-27.61,38.10,-0.03,196.0,-1.00,0.89,1.56,OX40N,NTT,DURABLES
5,APOLLOHOSP,8285.00,-13.86,55.0,H-LC,10.84,172216.0,16039.0,10057.0,-0.76,10.27,5.84,16.71,22.0,1.59,1.24,29.50,X40N,BTT,HEALTHCARE
27,GREENPANEL,537.00,236.56,65.0,M-SC,2.39,150182.0,-32896.0,111871.0,-0.71,-17.97,74.49,43.14,230.0,-0.29,1.08,41.27,XY24,NTT,MISC
32,HINDZINC,730.22,21.48,61.0,M-LC,1.37,198469.0,-6607.0,119181.0,-0.70,-3.22,60.05,54.89,52.0,-0.06,1.43,20.10,X5K,ATH,METALS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,104.70,58.0,M-SC,3.97,88196.0,-12581.0,12674.0,-0.52,-12.48,14.37,0.09,245.0,-0.99,0.64,15.87,OX40N,NTT,DURABLES
78,UJJIVANSFB,60.0,97.20,56.0,H-SC,9.54,124281.0,-18198.0,37719.0,0.61,-12.77,30.35,13.70,163.0,-0.48,0.90,46.69,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.06,64.0,H-SC,1.40,226980.0,-42687.0,79012.0,1.09,-15.83,34.81,13.47,138.0,-0.54,1.64,14.80,XY24,NTT,PAINTS
19,CERA,9475.0,-19.33,51.0,H-SC,2.08,147120.0,-28783.0,70809.0,0.35,-16.36,48.13,23.89,149.0,-0.41,1.06,27.06,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-19.01,56.0,H-MC,7.62,109592.0,-22243.0,64462.0,1.50,-16.87,58.82,32.03,98.0,-0.35,0.79,22.75,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-24.83,61.0,H-LC,12.32,299193.0,-46763.0,121622.0,0.32,-13.52,40.65,21.64,1.0,-0.38,2.16,5.99,X40,ATH,IT
40,INFY,2275.00,-15.29,59.0,H-LC,8.40,329468.0,16580.0,155114.0,1.60,5.30,47.08,54.87,3.0,0.11,2.38,12.30,X40,BTT,IT
42,ITC,452.00,-37.50,52.0,H-LC,2.52,201346.0,1208.0,19692.0,0.60,0.60,9.78,10.44,4.0,0.06,1.45,6.20,X40,NTT,FMCG
83,VBL,671.64,-17.03,39.0,H-LC,6.39,298584.0,-17258.0,129914.0,-0.65,-5.46,43.51,35.67,5.0,-0.13,2.15,7.63,X40N,ATH,FMCG
1,ABB,7934.00,-37.63,68.0,H-LC,10.34,260688.0,-931.0,120151.0,0.84,-0.36,46.09,45.57,7.0,-0.01,1.88,10.93,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,63.0,M-MC,6.96,227734.0,2772.0,164879.0,-0.38,1.23,72.40,74.53,192.0,0.02,1.64,33.88,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,14.38,66.0,H-MC,3.59,197400.0,32730.0,35236.0,1.74,19.88,17.85,41.28,89.0,0.93,1.42,29.45,X40N,ATH,AC
84,VOLTAS,1918.49,0.88,58.0,H-MC,2.48,211635.0,19893.0,76146.0,-0.07,10.37,35.98,50.08,99.0,0.26,1.53,17.88,XY25,ATH,AC
30,HAVELLS,2069.16,0.34,59.0,H-MC,1.88,248602.0,871.0,74183.0,0.01,0.35,29.84,30.30,92.0,0.01,1.79,13.69,X40,ATH,ELECTRICAL
18,CAMS,5211.76,0.16,66.0,H-SC,3.83,113758.0,11754.0,32171.0,1.22,11.52,28.28,43.06,122.0,0.37,0.82,30.61,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,65.0,H-MC,6.84,113309.0,12805.0,17835.0,0.25,12.74,15.74,30.49,79.0,0.72,0.82,49.27,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.41,52.0,M-SC,0.49,100688.0,9190.0,69565.0,0.07,10.04,69.09,86.07,223.0,0.13,0.73,46.22,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.84,42.0,H-SC,11.67,134270.0,10934.0,122830.0,-0.68,8.87,91.48,108.45,119.0,0.09,0.97,33.20,AR,ATH,MISC
86,WIPRO,420.00,-8.80,62.0,M-LC,5.62,160183.0,9238.0,100210.0,1.66,6.12,62.56,72.51,53.0,0.09,1.16,12.51,XR,NTT,IT
12,BANKINDIA,190.00,-27.95,67.0,H-MC,6.16,184668.0,4860.0,107551.0,0.36,2.70,58.24,62.52,88.0,0.05,1.33,37.71,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJESHEXPO,518.00,1896.21,67.0,L-SC,3.05,54455.0,-82722.0,82815.0,0.32,-60.30,152.08,0.07,267.0,-1.00,0.39,35.43,OX40N,NTT,JEWELLERY
7,ASIANTILES,137.00,6700.00,52.0,L-SC,5.18,74640.0,-19170.0,95786.0,0.74,-20.43,128.33,81.67,269.0,-0.20,0.54,44.54,XR,NTT,CERAMICS
22,COFFEEDAY,80.00,-43.66,52.0,L-SC,42.31,84864.0,-28685.0,68740.0,-0.29,-25.26,81.00,35.27,268.0,-0.42,0.61,107.71,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,12.74,89139.0,-11824.0,106352.0,-0.99,-11.71,119.31,93.63,148.0,-0.11,0.64,32.77,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.44,53.0,H-SC,12.99,94755.0,-3225.0,24826.0,-0.16,-3.29,26.20,22.05,152.0,-0.13,0.68,37.73,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3948.65,65.0,M-SC,5.79,126201.0,-21167.0,72982.0,2.12,-14.36,57.83,35.16,236.0,-0.29,0.91,28.90,XY24,NTT,AUTO
81,VAIBHAVGBL,521.0,67.49,73.0,H-SC,7.20,145286.0,-37489.0,148555.0,1.75,-20.51,102.25,60.77,125.0,-0.25,1.05,30.13,XR,NTT,JEWELLERY
22,COFFEEDAY,80.0,-43.66,52.0,L-SC,42.31,84864.0,-28685.0,68740.0,-0.29,-25.26,81.00,35.27,268.0,-0.42,0.61,107.71,XR,NTT,HOTELS
75,TITAGARH,1548.0,1.40,69.0,H-SC,3.26,166136.0,-25879.0,104765.0,0.76,-13.48,63.06,41.08,158.0,-0.25,1.20,40.78,XY24,NTT,ENGINEERING
54,RAJESHEXPO,518.0,1896.21,67.0,L-SC,3.05,54455.0,-82722.0,82815.0,0.32,-60.30,152.08,0.07,267.0,-1.00,0.39,35.43,OX40N,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.86
1,25,43.33
2,50,73.77


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.00
LC    33.01
MC    22.96
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.93
X40      15.23
X40N     13.02
XR       11.64
AR        8.66
XY25      8.14
OX40N     8.13
X200      1.81
MH        1.70
X5K       1.43
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.99
H-LC    26.34
H-MC    20.04
M-SC    13.47
M-LC     5.61
M-MC     2.62
L-SC     1.54
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.55,-11.99,73.05
FMCG,11.72,-4.26,40.73
FINANCE,9.62,-15.25,60.39
MISC,7.42,-11.87,74.09
BANKS,6.25,-15.02,75.53
PAINTS,5.80,-14.84,31.96
ELECTRICAL,5.68,-4.37,61.90
AC,3.66,4.60,27.71
INSURANCE,3.38,-7.28,44.08


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2834795.0,22
XR,1296236.0,14
AR,1295515.0,9
X40,735826.0,10
X40N,605275.0,10
OX40N,519160.0,10
XY25,394756.0,6
SR,243713.0,2
X5K,119181.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3391055.0,29
M-SC,1377887.0,17
H-LC,1222673.0,15
H-MC,1175393.0,15
M-LC,373068.0,4
M-MC,314279.0,2
L-SC,247341.0,3
L-MC,60379.0,1
L-LC,37096.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1020881.0      6
M-SC       XY24       796117.0      7
H-SC       AR         782951.0      5
           XR         749619.0      7
H-MC       XY24       489845.0      4
H-LC       X40        452168.0      5
           AR         351846.0      2
M-MC       XY24       314279.0      2
H-SC       X40N       284008.0      4
           OX40N      248084.0      4
           SR         243713.0      2
H-LC       X40N       225258.0      4
H-MC       X40        214058.0      4
           XY25       185633.0      2
L-SC       XR         164526.0      2
M-SC       AR         160718.0      2
M-LC       XY24       153677.0      2
M-SC       OX40N      123799.0      4
M-LC       X5K        119181.0      1
M-SC       XR         113951.0      2
           XY25       113702.0      1
H-MC       XR         107551.0      1
M-LC       XR         100210.0      1
H-MC       X40N        96009.0      2
L-SC       OX40N       82815.0      1
H-LC       X200        75080.0      1
M-SC       X40         69600.0      1
H-MC       OX40N       64462.0      1
H-SC       MH          61799.0      1
L-MC       XR          60379.0      1
H-LC       XY24        59996.0      1
           XY25        58325.0      2
L-LC       XY25        37096.0      1
H-MC       MH          17835.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
